# Explore the Store Business in Toronto

Introduction/Business Problem section:
    
Let's suppose that I'm operating a consulting firm, and recently I have a client which is a commercial group (family business) 
and its current business focuses on various stores across North America, such as Clothing Stores, Health Food Stores, Beer 
Stores, etc. The client/company wants to expand its business into Toronto city and my goal is to define the right business 
expansion strategy. What kind of business/store should the company consider in order to achieve the most possible success? 
Which district in Toronto city should the Store be located? 



Data section:

I will combine the DataFrame scraped from the webpage of Toronto city on wikipedia and the data about population census in 
each district of Toronto from Statistics Canada. The DataFrame is processed and grouped by Neighborhood of each district with sum
function. By comparing the numbers of each store type in Toronto city, I will know what is most popular at current business status
and what other types of store business is needed. By retrieve data based on Foursquare data API, I can get the most Common Store 
type in each district. By analyzing the population relationship with different combination of store type, I can define the store type in the district 